In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv("/content/sample_data/diabetes.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df.shape

(768, 9)

In [5]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [6]:
X= df.iloc[:,:-1].values
y= df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, random_state=42)

In [32]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [11]:
model= Sequential()
model.add(Dense(units=32,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.fit(X_train,y_train, batch_size=32, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6191 - loss: 0.6987 - val_accuracy: 0.6623 - val_loss: 0.6599
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6513 - loss: 0.6605 - val_accuracy: 0.6688 - val_loss: 0.6124
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6912 - loss: 0.6219 - val_accuracy: 0.6623 - val_loss: 0.5802
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6766 - loss: 0.6217 - val_accuracy: 0.7013 - val_loss: 0.5591
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6774 - loss: 0.5956 - val_accuracy: 0.7208 - val_loss: 0.5426
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7315 - loss: 0.5474 - val_accuracy: 0.7208 - val_loss: 0.5307
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7552 - loss: 0.5308 - val_accuracy: 0.7338 - val_loss: 0.5220
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7212 - loss: 0.5271 - val_accuracy: 0.

# 1.Select Appropriate optimizer
# 2. Select No. of nodes in layer
# 3. Select No. of layers
# 4. All in one model

In [13]:
# !pip install -U keras-tuner
import keras_tuner as kt

In [14]:
def build_model(hp):
  model= Sequential()
  model.add(Dense(32,activation='relu',input_dim=X_train.shape[1]))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [15]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [16]:
tuner.search(X_train,y_train,epochs=6,validation_data=(X_test,y_test))

In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7090 - loss: 0.5502 - val_accuracy: 0.7792 - val_loss: 0.5106
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7560 - loss: 0.5155 - val_accuracy: 0.7857 - val_loss: 0.5005
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7523 - loss: 0.5003 - val_accuracy: 0.7857 - val_loss: 0.4948
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7821 - loss: 0.4811 - val_accuracy: 0.7857 - val_loss: 0.4908
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7618 - loss: 0.5032 - val_accuracy: 0.7857 - val_loss: 0.4876
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7616 - loss: 0.4832 - val_accuracy: 0.7857 - val_loss: 0.4848
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7813 - loss: 0.4743 - val_accuracy: 0.7857 - val_loss: 0.4830
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7695 - loss: 0.4783 - val_accuracy: 0.78

In [21]:
def build_model(hp):
  model= Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=X_train.shape[1]))
  model.add(Dense(units=1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [22]:
tuner= kt.GridSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='my_dir',
                     project_name='rmspropss')

Reloading Tuner from my_dir/rmspropss/tuner0.json


In [23]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 8}

In [25]:
model= tuner.get_best_models(num_models=1)[0]

In [26]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,epochs=100)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7631 - loss: 0.5188   
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7374 - loss: 0.5293 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7488 - loss: 0.5030 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7403 - loss: 0.5309 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7577 - loss: 0.5084 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7551 - loss: 0.4960 
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7531 - loss: 0.5029 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7525 - loss: 0.4925 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7543 - loss: 0.5120 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7766 - loss: 0.4723 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7483 - loss: 0.4967 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [27]:
def build_models(hp):
  model= Sequential()

  model.add(Dense(8,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [28]:
kt.RandomSearch(build_models,
                objective='val_accuracy',
                max_trials=10,
                directory='my_dir',
                project_name='layer_nm')

In [29]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [30]:
tuner.get_best_hyperparameters()[0].values

{'units': 8}

In [33]:
def build_mogdel(hp):
  model= Sequential()

  counter=0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8, max_value=128, step=8),activation=hp.Choice('activation',['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8, max_value=128, step=8),activation=hp.Choice('activation',['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',['adam','adamaw','nadam','rmsprop','adadelta','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
  return model

In [34]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=10,
                       directory='my_dir',
                       project_name='whole_prop')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 42s


In [38]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
model.fit(X_train,y_train,epochs=200,initial_epoch=10,validation_data=(X_test,y_test))

Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7760 - loss: 0.4864 - val_accuracy: 0.7857 - val_loss: 0.4889
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7765 - loss: 0.4611 - val_accuracy: 0.7857 - val_loss: 0.4863
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7749 - loss: 0.4670 - val_accuracy: 0.7922 - val_loss: 0.4832
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7910 - loss: 0.4277 - val_accuracy: 0.7857 - val_loss: 0.4835
Epoch 15/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7986 - loss: 0.4429 - val_accuracy: 0.7792 - val_loss: 0.4855
Epoch 16/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7846 - loss: 0.4641 - val_accuracy: 0.7857 - val_loss: 0.4880
Epoch 17/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7878 - loss: 0.4412 - val_accuracy: 0.7857 - val_loss: 0.4910
Epoch 18/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7876 - loss: 0.4574 - val_accuracy: 0